In [1]:
import time
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import urllib3
import requests

In [2]:
# Call the dataset
arxiv_links = pd.read_excel("./query-links/Arxiv_Query_Links.xlsx")

In [4]:
# Open the drive
driver = uc.Chrome()

In [ ]:
# Get the paper links
list_source = []
list_querystring = []
list_querylinks = []
list_paperlinks = []
for index in range(0,len(arxiv_links["Query Links"])):
  driver.get(arxiv_links["Query Links"][index])
  time.sleep(5)
  try:
    # Get links from the first page
    hrefs = driver.find_elements_by_xpath("//*[@class='list-title is-inline-block']//*[@href]")
    for a in hrefs:
        link = a.get_attribute('href')
        if "abs" in link:
            list_source.append("Arxiv")
            list_querystring.append(arxiv_links["Query"][index])
            list_querylinks.append(arxiv_links["Query Links"][index])
            list_paperlinks.append(link)

    # Get links from the next page
    while True:
        try:
            driver.find_element_by_class_name("pagination-next").click()
            time.sleep(5)
            hrefs = driver.find_elements_by_xpath("//*[@class='list-title is-inline-block']//*[@href]")
            for a in hrefs:
                link = a.get_attribute('href')
                if "abs" in link:
                    list_source.append("Arxiv")
                    list_querystring.append(arxiv_links["Query"][index])
                    list_querylinks.append(arxiv_links["Query Links"][index])
                    list_paperlinks.append(link)
        except:
            break
  except:
    list_source.append("Arxiv")
    list_querystring.append(arxiv_links["Query"][index])
    list_querylinks.append(arxiv_links["Query Links"][index])
    list_paperlinks.append("no_result")

In [6]:
driver.quit()

In [ ]:
# Save to xlsx
arxiv_paperlinks = pd.DataFrame({'Source': list_source,'Query': list_querystring, 'Query Links': list_querylinks, 'Paper Links': list_paperlinks})
arxiv_paperlinks.to_excel("./paper-links/PaperLinks_Arxiv_AllDuplicated.xlsx", index = False)

In [ ]:
# Drop Duplicate
arxiv_paperlinks_dd = arxiv_paperlinks.drop_duplicates(['Paper Links'], keep='first')
arxiv_paperlinks_dd.to_excel("./paper-links/PaperLinks_Arxiv.xlsx", index = False)